In [91]:
from azureml.core import Workspace
ws=Workspace.from_config()

2022-10-17:03:50:08,767 INFO     [workspace.py:291] Found the config file in: /config.json
2022-10-17:03:50:08,801 INFO     [_universal.py:476] Request URL: 'https://management.azure.com/subscriptions?api-version=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': 'cb50381e-4dce-11ed-ab33-a5473f9416ea'
    'User-Agent': 'azsdk-python-azure-mgmt-resource/21.1.0 Python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10)'
    'Authorization': 'REDACTED'
No body was attached to the request
2022-10-17:03:50:08,858 INFO     [_universal.py:504] Response status: 200
Response headers:
    'Cache-Control': 'no-cache'
    'Pragma': 'no-cache'
    'Content-Type': 'application/json; charset=utf-8'
    'Content-Encoding': 'REDACTED'
    'Expires': '-1'
    'Vary': 'REDACTED'
    'x-ms-ratelimit-remaining-tenant-reads': '11999'
    'x-ms-request-id': '510a5250-15d4-4c5a-91a2-843ae6975f65'
    'x-ms-correlation-request-id': 'REDACTED'
    '

In [92]:
pipeline_cluster = "Demo-Compute-Cluster"
compute_target = ws.compute_targets[pipeline_cluster]

In [93]:
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration

curated_env=Environment.get(workspace=ws, name='AzureML-sklearn-1.0-ubuntu20.04-py38-cpu')
pipeline_run_config=RunConfiguration()
pipeline_run_config.environment=curated_env

In [94]:
import pandas as pd

data = pd.read_csv(
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
)
data=data.head(100)

In [95]:
from azureml.core import Dataset,Datastore
from azureml.data.datapath import DataPath

datastore_name= ws.get_default_datastore()
from azureml.data.dataset_factory import TabularDatasetFactory

inference_data = TabularDatasetFactory.register_pandas_dataframe(
                                                data, 
                                                target=(datastore_name, "scoring"), 
                                                name="inference_Classification_dataset",
                                                description='Data for Inference', 
                                                tags={ 'type': 'TabularDataset' }
)

2022-10-17:03:50:23,926 INFO     [datastore_client.py:991] <azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f57ea1b56d0>


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to scoring/761d4351-018f-4c26-9b05-9012f47dc628/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [96]:
df=inference_data.to_pandas_dataframe()
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.80,92.89,-46.20,1.30,5099.10,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.80,92.89,-46.20,1.31,5099.10,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.40,94.47,-41.80,4.97,5228.10,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.40,93.92,-42.70,4.96,5228.10,no


In [97]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep

#OutputFileDatasetConfig with giving destination
output_path = (datastore_name, f"azureml/classification_inference_prep_output/")
prepped_output_path = OutputFileDatasetConfig(destination = output_path)

input_ds = Dataset.get_by_name(ws, 'inference_Classification_dataset')

prep_step=PythonScriptStep(
    name="Prepare Data for Inference",
    script_name="prepare.py",
    source_directory="./Scripts",
    arguments=["--output_path",prepped_output_path],
    inputs=[input_ds.as_named_input('inference_Classification_dataset')],
    compute_target=compute_target,
    runconfig=pipeline_run_config
)

In [98]:
prepped_inference_data=prepped_output_path.read_delimited_files("prepped_inference_data_classification.csv") # This is the data to be used for scoring

# 

In [101]:
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.data import OutputFileDatasetConfig

output_path = (datastore_name, f"azureml/classification_scoring_results/")
inference_output_path = OutputFileDatasetConfig(destination = output_path)

parallel_run_config = ParallelRunConfig(
    source_directory="./Scripts",
    entry_script="score.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=curated_env,
    compute_target=compute_target,
    node_count=2)

parallelrun_step = ParallelRunStep(
    name='batch-scoring',
    parallel_run_config=parallel_run_config,
    inputs=[prepped_inference_data],
    output=inference_output_path,
    arguments=['--model_name','automlmodel'],
    allow_reuse=False
)

print('Steps defined')

Steps defined


In [102]:
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment

experiment = Experiment(ws, name= "classification-E2E_Inference_Pipeline")

pipeline = Pipeline(ws, [prep_step,parallelrun_step])

pipeline_run = experiment.submit(pipeline, show_output=True)
pipeline_run.wait_for_completion()

2022-10-17:04:06:49,458 INFO     [datastore_client.py:991] <azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f57ea1b56d0>


Created step Prepare Data for Inference [9a3f31d4][dbf8fbdf-15f5-4319-be31-a4bbddd365b7], (This step will run and generate new outputs)
Created step batch-scoring [5ee1b620][92a71f06-d2b2-465c-b060-a848e675f262], (This step will run and generate new outputs)
Submitted PipelineRun 2c87de25-f7af-457d-a873-3e857744f63b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/2c87de25-f7af-457d-a873-3e857744f63b?wsid=/subscriptions/fe38c376-b42a-4741-9e7c-f5d7c31e5873/resourcegroups/prodrg/workspaces/prod&tid=16b3c013-d300-468d-ac64-7eda0820b6d3
PipelineRunId: 2c87de25-f7af-457d-a873-3e857744f63b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/2c87de25-f7af-457d-a873-3e857744f63b?wsid=/subscriptions/fe38c376-b42a-4741-9e7c-f5d7c31e5873/resourcegroups/prodrg/workspaces/prod&tid=16b3c013-d300-468d-ac64-7eda0820b6d3
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 5de92f54-3e82-4616-932d-49adbb27c6a0
Link to Azure Machine Learning Portal: https:

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User program failed with Exception: Run failed, please check logs for details. You can check logs/readme.txt for the layout of logs.",
        "messageParameters": {},
        "detailsUri": "https://aka.ms/azureml-run-troubleshooting",
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"User program failed with Exception: Run failed, please check logs for details. You can check logs/readme.txt for the layout of logs.\",\n        \"messageParameters\": {},\n        \"detailsUri\": \"https://aka.ms/azureml-run-troubleshooting\",\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

In [ ]:
'''Put sample scoring step for time series'''

In [6]:
model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/...
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('34', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=True, with_std=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.1, learning_rate=0.03158578947368421, max_bin=140, max_depth=3, min_child_weight=2, min_data_in_leaf=0.027593448275862072, min_split_gain=0.9473684210526315, n_estimators=400, n_jobs=-1, num_leaves=5, pr

batch_pipeline


In [12]:
%%writefile $Scripts/scoring.py
import os
import numpy as np
from azureml.core import Model
import joblib


def init():
    # Runs when the pipeline step is initialized
    global model

    # load the model
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)


def run(mini_batch):
    # This runs for each batch
    resultList = []

    # process each file in the batch
    for f in mini_batch:
        # Read the comma-delimited data into an array
        data = np.genfromtxt(f, delimiter=',')
        # Reshape into a 2-dimensional array for prediction (model expects multiple items)
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

Writing batch_pipeline/batch_diabetes.py


In [13]:
%%writefile $experiment_folder/batch_environment.yml
name: batch_environment
dependencies:
- python=3.6.2
- scikit-learn
- pip
- pip:
  - azureml-defaults

Writing batch_pipeline/batch_environment.yml


In [14]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# Create an Environment for the experiment
batch_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/batch_environment.yml")
batch_env.docker.base_image = DEFAULT_CPU_IMAGE
print('Configuration ready.')

Configuration ready.


In [18]:
from azureml.core import Datastore, Dataset
import pandas as pd
import os

# Set default data store
ws.set_default_datastore('workspaceblobstore')
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

# Load the diabetes data
diabetes = pd.read_csv('data/diabetes2.csv')
#diabetes['BMI']=round(diabetes['BMI'])
#diabetes['DiabetesPedigree']=round(diabetes['DiabetesPedigree'])
# Get a 2-item sample of the feature columns (not the diabetic label)
sample = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].sample(n=2).values
print(sample)

azureml_globaldatasets - Default = False
workspacefilestore - Default = False
workspaceworkingdirectory - Default = False
workspaceblobstore - Default = True
workspaceartifactstore - Default = False
[[  8  92  93  47  36  21   1  23]
 [  0 171  80  34  23  43   1  21]]


In [ ]:
# new register_dataset.py
import argparse
from azureml.core import Workspace, Dataset

def parse_args():
    parser = argparse.ArgumentParser(description="Register dataset")
    parser.add_argument("-n", type=str, help="Name of the dataset you want to register")
    parser.add_argument("-d", type=str, help="Description of the dataset you want to register")
    parser.add_argument("-t", type=str, help="type of dataset", default='local')    
    parser.add_argument("-l", type=str, help="local path of the dataset folder", default='data/')
    parser.add_argument("-p", type=str, help="Path on data store", default='data/')
    parser.add_argument("-s", type=str, help="Storage url for cloud storage")
    return parser.parse_args()

def main():
    args = parse_args()
    print(args)
    ws = Workspace.from_config()
    if args.t == "local":
        print("local data")
        datastore = ws.get_default_datastore()
        datastore.upload(src_dir = args.l, target_path = args.p, overwrite = True, show_progress = True)
        print(f"About to register dataset {args.n}")

        dataset = Dataset.File.from_files(path=[(datastore, args.p)], validate=True)
        dataset = dataset.register(workspace=ws,name=args.n, description=args.d,create_new_version=True)
        print("Dataset registered")
    else:
        print("cloud data")
        data_urls = [args.s]
        dataset = Dataset.File.from_files(data_urls)
        dataset = dataset.register(workspace=ws,name=args.n, description=args.d,create_new_version=True)
    

if __name__ == "__main__":
    main()

Steps defined


In [21]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
pipeline_run = Experiment(ws, 'mslearn-diabetes-batch').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)



Created step batch-score-diabetes [204064bc][65c039e3-182a-48ee-ba32-6a21f0885367], (This step will run and generate new outputs)
Submitted PipelineRun 6ddc4b06-fdef-426d-a46b-bebaf785d187
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/6ddc4b06-fdef-426d-a46b-bebaf785d187?wsid=/subscriptions/1a15a546-9f17-4a32-a1b6-da7850ab3d04/resourcegroups/machinelearningexperiments/workspaces/ml-workspace&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRunId: 6ddc4b06-fdef-426d-a46b-bebaf785d187
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/6ddc4b06-fdef-426d-a46b-bebaf785d187?wsid=/subscriptions/1a15a546-9f17-4a32-a1b6-da7850ab3d04/resourcegroups/machinelearningexperiments/workspaces/ml-workspace&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 4742184a-632f-4e2e-9332-700dbcb91e5a
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/4742184a-632f-4e2e-9332-700dbcb91e5a?wsid=/subscripti

'Finished'

In [22]:
published_pipeline = pipeline_run.publish_pipeline(
    name='diabetes-batch-pipeline', description='Batch scoring of diabetes data', version='1.0')

published_pipeline

Name,Id,Status,Endpoint
diabetes-batch-pipeline,9c2b7357-2dd0-45b6-a2e3-c9a8e672ed97,Active,REST Endpoint


In [23]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://uksouth.api.azureml.ms/pipelines/v1.0/subscriptions/1a15a546-9f17-4a32-a1b6-da7850ab3d04/resourceGroups/machinelearningexperiments/providers/Microsoft.MachineLearningServices/workspaces/ml-workspace/PipelineRuns/PipelineSubmit/9c2b7357-2dd0-45b6-a2e3-c9a8e672ed97


In [24]:
from azureml.pipeline.core import Schedule,ScheduleRecurrence

weekly=ScheduleRecurrence(frequency='Week',interval=1)
pipeline_schedule=Schedule.create(ws,
name='weekly predictions',
pipeline_id=published_pipeline.id,
experiment_name='Batch_predictions',
recurrence=weekly)

In [ ]:
#deploy_batch_scoring_pipeline.py
ws = Workspace.from_config()
env = Environment.get(workspace=ws, name=config['batch_env_name'])
runconfig = RunConfiguration()
runconfig.environment = env

# Dataset input and output
batch_dataset = Dataset.get_by_name(ws, config['batch_input_dataset_name'])
batch_dataset_parameter = PipelineParameter(name="batch_dataset", default_value=batch_dataset)
batch_dataset_consumption = DatasetConsumptionConfig("batch_dataset", batch_dataset_parameter).as_download()

datastore = ws.get_default_datastore()
output_dataset = OutputFileDatasetConfig(name='batch_results',
                                         destination=(datastore, config['batch_output_path_on_datastore'])).register_on_complete(name=config['batch_output_dataset_name'])

parallel_run_config = ParallelRunConfig(
    source_directory="data-science/src/",
    entry_script="score.py",
    environment=env,
    output_action="append_row",
    append_row_file_name=config['batch_output_filename'],
    mini_batch_size=config['batch_mini_batch_size'],
    error_threshold=config['batch_error_threshold'],
    compute_target=config['batch_target'],
    process_count_per_node=config['batch_process_count_per_node'],
    node_count=config['batch_node_count']
)

batch_step = ParallelRunStep(
    name="batch-step",
    parallel_run_config=parallel_run_config,
    arguments=['--model_name', config['model_name'], '--enable_monitoring', args.m, '--table_name', config["scoring_table_name"]],
    inputs=[batch_dataset_consumption],
    side_inputs=[],
    output=output_dataset,
    allow_reuse=False
)

steps = [batch_step]

print('Creating, validating, and publishing pipeline')
#print('Creating, validating, and publishing pipeline')
#pipeline = Pipeline(workspace=ws, steps=steps)
#pipeline.validate()
#published_pipeline = pipeline.publish(config['training_pipeline_name'])

In [ ]:
#add pipeline to an endpoint
import argparse
import azureml.core
from azureml.core import Workspace
from azureml.pipeline.core import PublishedPipeline, PipelineEndpoint

print("Azure ML SDK version:", azureml.core.VERSION)

parser = argparse.ArgumentParser("Add pipeline to pipeline endpoint")
parser.add_argument("-p", type=str, help="Pipeline ID of pipeline to add")
args = parser.parse_args()

ws = Workspace.from_config()

pipeline_id = args.p
published_pipeline = PublishedPipeline.get(workspace=ws, id=pipeline_id)
endpoint_name = published_pipeline.name + '-endpoint'

try:
    pl_endpoint = PipelineEndpoint.get(workspace=ws, name=endpoint_name)
    pl_endpoint.add_default(published_pipeline)
    print(f'Added pipeline {pipeline_id} to Pipeline Endpoint with name {endpoint_name}')
except Exception:
    print(f'Will create new Pipeline Endpoint with name {endpoint_name} with pipeline {pipeline_id}')
    pl_endpoint = PipelineEndpoint.publish(workspace=ws,
                                           name=endpoint_name,
                                           pipeline=published_pipeline,
                                           description='Published by MLOps V2')

#run_pipeline.py
import argparse
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.pipeline.core import PublishedPipeline

print("Azure ML SDK version:", azureml.core.VERSION)

parser = argparse.ArgumentParser("Run Pipeline")
parser.add_argument("-p", type=str, help="Pipeline Id")
args = parser.parse_args()

ws = Workspace.from_config()

pipeline = PublishedPipeline.get(workspace=ws, id=args.p)
experiment_name = pipeline.name + '-ci'
pipeline_run = Experiment(ws, experiment_name).submit(pipeline, regenerate_outputs=True)
print(pipeline_run)
pipeline_run.wait_for_completion()

In [ ]:
Once the inference pipeline is created, we can deploy the model to an endpoint